# OrgAn
*Raphaël Tisseyre, Bastien Pinsard, Johan Schmidt*

#### Introduction

#### pchem_rq.py

#### functions.py

#### chromato.py

This file contains a series of functions useful for HPLC applications. 

The function `get_elution_order` takes a DataFrame of several compounds as an input, and sorts them by their logP (ascending if it’s normal phase, descending if it’s reverse phase) to get an elution order estimate. Since the logP is nothing more than the partition coefficient between octanol and water, and that the retention factor is the partition coefficient between the mobile and stationary phases, the logP can give an estimate of the elution order. However, this estimate doesn’t take into account other factors, such as dipole moment and hydrogen bonds. 

The function `calculate_polarity_index` calculates the polarity index of an eluant using the table 28.4.1 provided [here](https://chem.libretexts.org/Bookshelves/Analytical_Chemistry/Instrumental_Analysis_(LibreTexts)/28%3A_High-Performance_Liquid_Chromatography/28.04%3A_Partition_Chromatography).  

In [12]:
from src.OrgAn.chromato import calculate_polarity_index

print(calculate_polarity_index(n_hex = 0.4, etoac = 0.6))

2.68


The function `estimate_retention_factor` estimates a retention factor based on a logP value. The retention factor is calculated using the formula $ \log k = \log P – 0.5 \cdot (10.2 - P’) $, where $ \log k $ is the log of the retention factor and $ P’ $ is the polarity index of the eluant. As said before, the retention factor is correlated with the logP, as they are both partition coefficients. The [article](https://chem.libretexts.org/Bookshelves/Analytical_Chemistry/Instrumental_Analysis_(LibreTexts)/28%3A_High-Performance_Liquid_Chromatography/28.04%3A_Partition_Chromatography) provided above states: 
> As a general rule, a two unit change in the polarity index corresponds to an approximately 10-fold change in a solute’s retention factor. Here is a simple example. If a solute’s retention factor, k, is 22 when using water as a mobile phase (P′ = 10.2), then switching to a mobile phase of 60:40 water–methanol (P′ = 8.2) decreases k to approximately 2.2.

Using these values, we can find an equation to approximate the retention factor based on the logP. 

#### Conclusion